# Imports

In [1]:
import os

from tqdm import tqdm

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader

from src.utils.preprocessing import Preprocessor
from src.utils.dataset import ForestDataset
from src.models.unet import UNet

from src.utils.loss import loss as criterion

# Notebook settings

In [2]:
# %load_ext tensorboard
# %tensorboard --logdir logs
# %matplotlib inline

# Global parameters

In [4]:
IMG_DIR = "data/images"
GEDI_PATH = "data/gedi/gedi_complete.fth"
LABELS_DIR = "data/train"
PATCH_INFO_FILE = "data/info/patches.fth"
LOGS_DIR = "logs"
MODEL_DIR = "models"
BATCH_SIZE = 64
NUM_WORKERS = 6
LEARNING_RATE = 1e-4
EPOCHS = 2
DEVICE = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {DEVICE} device")

Using mps device


# Preprocess labels and patches

| image | patch | n_labels |
| - | - | - | 
| L15-1059E-1355N	 | 0	 | 28 |

In [5]:
preprocessor = Preprocessor(
    patch_info_file=PATCH_INFO_FILE,
    img_dir=IMG_DIR,
    labels_dir=LABELS_DIR,
    gedi_file=GEDI_PATH,
)

preprocessor.run()

patches = preprocessor.patch_info

print(f"Total number of patches: {len(patches)}")

INFO:root:Starting preprocessing...
INFO:root:Directories validated.
INFO:root:Images loaded.
INFO:root:Loaded existing patch info file. Skipping image processing.


53308

## Filter patches

In [6]:
# TODO

# Create training & validation splits

In [7]:
# TODO

## Create dataset & dataloader

In [8]:
dataset = ForestDataset(
    img_dir=IMG_DIR,
    labels_dir=LABELS_DIR,
    patches=patches,
)

dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

# Create U-Net model

In [9]:
# Initialize our model
model = UNet().to(DEVICE)

# Specify a loss function and an optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Create a tensorboard writer
writer = SummaryWriter(LOGS_DIR)

# Train U-Net model

In [10]:
# Training loop
for epoch in range(EPOCHS):
    for i, (inputs, targets) in enumerate(tqdm(dataloader, desc=f"Epoch {epoch+1}")):
        # Move inputs and targets to the appropriate device
        inputs = inputs.to(DEVICE)
        targets = targets.to(DEVICE)

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Log the loss value to TensorBoard
        writer.add_scalar("loss/train", loss, epoch * len(dataloader) + i)

    # Print loss every epoch
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {loss.item()}")


# Close the SummaryWriter when you're done
writer.close()

Epoch 1:   0%|          | 0/833 [00:00<?, ?it/s]/Users/ematala/dev/tree-heights-germany/.venv/lib/python3.11/site-packages/torch/autograd/__init__.py:200: UserWarning: The operator 'aten::sgn.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Epoch 1: 100%|██████████| 833/833 [07:13<00:00,  1.92it/s]


Epoch [1/2], Loss: 6.268318176269531


Epoch 2: 100%|██████████| 833/833 [07:06<00:00,  1.95it/s]

Epoch [2/2], Loss: 5.314236640930176


# Export model

In [11]:
torch.save(model.state_dict(), os.path.join(MODEL_DIR, "unet.pt"))

# Load pre-trained model

In [12]:
# model = load_model(model, "models/unet.pt", DEVICE)
# model

# Visualise results

In [13]:
# predict_image(model, "data/images/L15-1059E-1348N.tif", DEVICE)

In [14]:
# np.mean(prediction[prediction > 0.5])

In [15]:
# plot_image("data/images/L15-1059E-1348N.tif")